In [1]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [2]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
#gis = GIS(r"https://adsrv2019.ad.local/portal", username, password)
gis = GIS(r"https://ebase.ad.local/portal", username=username, password=password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Get list of part names to process

In [3]:
partNameQuery = """MATCH (p:Product)
WITH DISTINCT p.Parts as parts
RETURN parts"""
x=knowledge_graph.query(partNameQuery)
y = []
partNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        partNames.append(s)
partNames = list(set(partNames))
print(partNames)

['Part 3', 'Part 5', 'Part 7', 'Part 2', 'Part 6', 'Part 4', 'Part 1']


### Query

In [4]:
def submitQuery(partName):
    opQuery = """MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "{0}")
    MATCH (part:Part)
    WHERE part.PartName = "{0}"
    RETURN product, part""".format(partName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [5]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProductName'])
        print(d['_properties']['PartName'])
        print(d['_id'])
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "UsedIn",
            "_originEntityId": UUID(f"{d['_id']}"), # part ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [6]:
for part in partNames:
    print(f"Finding related Products for {part}")
    queryNodes = submitQuery(part)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Products for Part 3
MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "Part 3")
    MATCH (part:Part)
    WHERE part.PartName = "Part 3"
    RETURN product, part
Related nodes:
Product 1
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
Related nodes:
Product 2
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
Related nodes:
Product 3
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
Related nodes:
Product 5
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
Related nodes:
Product 7
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
Related nodes:
Product 14
Part 3
91dce121-8f88-4d2d-a45b-c754f20665f5
All done! 6 relationships created.
Finding related Products for Part 5
MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "Part 5")
    MATCH (part:Part)
    WHERE part.PartName = "Part 5"
    RETURN product, part
Related nodes:
Product 5
Part 5
8418d656-4ba5-4629-97aa-e34b169e7210
Related nodes:
Product 6
Part 5
8418d656-4ba5-4629-97